# 0. Initialize

## 0.1. Import Libraries

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

## 0.2. DEFINE VARIABLES 

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
DATA_PATH = '/content/drive/MyDrive/raw_data/' # '<insert-your-training-data-path-here>'

ROUND = 2 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '26555'#'<insert-your-id-here>'
PROJECT_CODE = 'CS4121c3b6284da3e'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx


import os

# Use getcwd() function  
pathSpecified = os.getcwd()  
# Using listdir() function  
listOfFileNames = os.listdir(pathSpecified)  
# Print the name of all files in the current working directory  
print("Following is the list of names of all the files present in the current working directory: ")  
print(listOfFileNames)  



Following is the list of names of all the files present in the current working directory: 
['.config', 'drive', 'sample_data']


## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [ ]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
2995,1593539327623151619,Yes
2996,1393886554062524418,No
2997,1597925615092764672,Yes
2998,1585291418616176640,Yes


In [ ]:
ANNOT_PATH = '/content/drive/MyDrive/MyData/'

annotation_tweet = pd.read_csv(('{}annotated_tweets.csv').format(ANNOT_PATH), dtype={'Unnamed: 0': str, 'isPolitical': str})

annotation_tweet.drop(['url', 'sentiment', 'isExperiential', 'isInsult'], 1, inplace=True)
annotation_tweet.rename(columns = {'Unnamed: 0':'tweet_id'}, inplace=True)

# Concatenating -> droping duplicates -> reindexing
trainingTweetDf = pd.concat([annotation_tweet, trainingTweetDf], axis='rows')
trainingTweetDf.drop_duplicates(subset=['tweet_id'], inplace=True)
trainingTweetDf.reset_index(drop=True, inplace=True)

trainingTweetDf

<ipython-input-6-af2ca0d07865>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  annotation_tweet.drop(['url', 'sentiment', 'isExperiential', 'isInsult'], 1, inplace=True)


,tweet_id,isPolitical,topics
0,1588534866848927747,Yes,['Unemployment']
1,1384975295107764228,NaN,NaN
2,1577024342268837888,Yes,['Crime&justice']
3,1352747611623714816,No,['Music&TV']
4,1588246726296469504,Yes,['Unemployment']
...,...,...,...
3195,1593539327623151619,Yes,NaN
3196,1393886554062524418,No,NaN
3197,1597925615092764672,Yes,NaN
3198,1585291418616176640,Yes,NaN


In [ ]:
trainingTweetDf.isPolitical.value_counts()

Yes    2100
No     1097
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [ ]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
2995,djblumenberg,No
2996,mel1sq,No
2997,eren_yz1,Yes
2998,ergnyildiz4,No


In [ ]:
annotation_user = pd.read_csv(('{}annotated_users.csv').format(ANNOT_PATH), dtype={'Unnamed: 0': str, 'isBot': str})

annotation_user.drop(['url', 'isOrganizational', 'isTroll'], 1, inplace=True)
annotation_user.rename(columns = {'Unnamed: 0':'screen_name'}, inplace=True)

# Concatenating -> droping duplicates -> reindexing
trainingUserDf = pd.concat([annotation_user, trainingUserDf], axis='rows')
trainingUserDf.drop_duplicates(subset=['screen_name'], inplace=True)
trainingUserDf.reset_index(drop=True, inplace=True)

trainingUserDf

<ipython-input-9-6ca9e12bea23>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  annotation_user.drop(['url', 'isOrganizational', 'isTroll'], 1, inplace=True)


,screen_name,isBot,gender
0,qara118,Yes,Female
1,mhmtbyndrw,Yes,Male
2,hasansimsek003,Not sure,Male
3,celebimehmeta,No,Male
4,cryptowisky,No,Not sure
...,...,...,...
3195,djblumenberg,No,NaN
3196,mel1sq,No,NaN
3197,eren_yz1,Yes,NaN
3198,ergnyildiz4,No,NaN


In [ ]:
trainingUserDf.isBot.value_counts()

No          2577
Yes          617
Not sure       6
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [ ]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS4121c3b6284da3e.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [ ]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:/Users/suuser/Desktop/Sabancı/CS412/spring-2022/project/'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [ ]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [ ]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [ ]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [ ]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [ ]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [ ]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

#### (New Func 1) 1.1.1.7. Check If Tweet is a Reply

In [ ]:
def check_if_reply(tweet_metadata_line):
    text = tweet_metadata_line['text']
    if text[0] == '@':
      return 1
    else:
      return 0


#### (New Func 2) 1.1.1.8. Get Frequent Tweet Sources

In [ ]:
def get_frequent_tweet_source(tweet_metadata_line):
    # Initialize variables
    android_count = 0
    iphone_count = 0
    other_count = 0
    
    for tweet in tweet_metadata_line:
        tweet_source = tweet_metadata_line['source']
        if 'android' in tweet_source.lower():
            android_count += 1
        elif 'iphone' in tweet_source.lower():
            iphone_count += 1
        else:
            other_count += 1
    
    # Find the device with the highest count
    if android_count > iphone_count and android_count > other_count:
        return 'Android'
    elif iphone_count > android_count and iphone_count > other_count:
        return 'iPhone'
    else:
        return 'Other'

#### (New Func 3) 1.1.1.9. Get Acc Age

In [ ]:
from datetime import datetime

def get_age(tweet_metadata_line):
    str_age = tweet_metadata_line['user']['created_at'] 
    age = datetime.strptime(str_age, '%a %b %d %H:%M:%S %z %Y')

    return age

#### (New Func 4) 1.1.1.9.1 Get Number of Tweets 

In [ ]:
def get_tweet_count(tweet_metadata_line): 
    tweets = tweet_metadata_line['user']['statuses_count']
    return tweets

#### ROUND2 ADDITIONS

In [ ]:
def count_digits_in_username(tweet_metadata_line):
    user = tweet_metadata_line.get('user', {})
    screen_name = user.get('screen_name')
    if screen_name:
        digit_count = sum(c.isdigit() for c in screen_name)
    else:
        digit_count = 0
    return digit_count


In [ ]:
def get_tweet_length(tweet_metadata_line):
    tweet = tweet_metadata_line.get('extended_tweet',{}).get('full_text', tweet_metadata_line.get('text', ''))
    tweet_length = len(tweet)
    return tweet_length


In [ ]:
from datetime import datetime
import pytz

def get_average_fav(tweet_metadata_line):
    user = tweet_metadata_line.get('user', {})
    favourites_count = user.get('favourites_count')
    created_at = user.get('created_at')
    if created_at:
        account_age_days = (datetime.now(pytz.UTC) - datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y')).days
    else:
        account_age_days = 0
    if favourites_count and account_age_days:
        average_fav = favourites_count / account_age_days
    else:
        average_fav = 0
    return average_fav


### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [ ]:
def check_political_ent(text, checkIn):
    text = text.lower()

    tweet_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag',
                        'eczozgurozel', 'ekrem_imamoglu', 'mansuryavas06', 'suleymansoylu', 'T_Karamollaoglu', 'DurmusYillmaz',
                        'erkbas', 'TCAytunCiray', 'tuncsoyer', 'dbdevletbahceli', 'bybekirbozdag', 'NureddinNebati', 'mkulunk',
                        'celebimehmeta', 'drfahrettinkoca', 'alibabacan', 'ahmet_davutoglu', 'erbakanfatih']
    retweet_from_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag',
                        'eczozgurozel', 'ekrem_imamoglu', 'mansuryavas06', 'suleymansoylu', 'T_Karamollaoglu', 'DurmusYillmaz',
                        'erkbas', 'TCAytunCiray', 'tuncsoyer', 'dbdevletbahceli', 'bybekirbozdag', 'NureddinNebati', 'mkulunk',
                        'celebimehmeta', 'drfahrettinkoca', 'alibabacan', 'ahmet_davutoglu', 'erbakanfatih']
                        
    entities_in_text = []

    if checkIn == 'retweetedFrom':
        entities_in_text = [ent for ent in retweet_from_entities if ent.lower() in text]
        if len(entities_in_text) >= 1:
            return 1

    elif checkIn == 'tweet':
      entities_in_text = [ent for ent in retweet_from_entities+tweet_entities if ent.lower() in text]

    return len(entities_in_text)

#### 1.1.2.2. Number of total interactions

In [ ]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

#### (New Func) 1.1.2.3. Check for political party names

In [ ]:
def check_political_parties(text):
    if text is None:
        return 0
    
    list_of_parties = ['AKP', 'CHP', 'MHP', 'HDP', 'IYI', 'DP', 'ak parti']
    
    parties_in_text = []
    for party in list_of_parties:
        # Split the text into a list of words
        words = text.split()
        
        # Check if the party name appears as a standalone word in the list of words
        if party.lower() in [word.lower() for word in words]:
            parties_in_text.append(party)
    
    number_parties = len(parties_in_text)
    return number_parties


#### (New Func 3) 1.1.2.5. Check Tweets Per Day

In [ ]:
import pytz

def TweetsPerDay(tweetCount, age):
    now = datetime.now(pytz.UTC)
    deltaTime = now-age
    return tweetCount/deltaTime.days

#### (New Func 4) 1.1.2.6. Check Political Word Ratio

In [ ]:
def political_word_total_word_ratio(text):
    num_political_entities = check_political_ent(text, 'tweet')
    total_word = len(set(text.split()))

    return num_political_entities/total_word

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [ ]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'entities_in_text':[],
              'total_interactions':[],
              'is_reply':[],
              'source_counts':[],
              'account_activeness':[],
              'account_age':[],
              'political_word_ratio':[],
              'digit_count':[],
              'tweet_length':[],
              'avg_fav':[],
              'tweets_per_day':[]
              }


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)
        is_reply = check_if_reply(line) 
        source_counts = get_frequent_tweet_source(line) 
        account_age = get_age(line) 
        tweets_count = get_tweet_count(line) 
        digit_count = count_digits_in_username(line)
        tweet_length = get_tweet_length(line)
        avg_fav = get_average_fav(line)
        
  

        # manually crafted data:
        entities_in_text = check_political_ent(text, 'tweet')
        total_num_interactions = total_interactions(retweet_count, favorite_count)             
        account_activeness = TweetsPerDay(tweets_count, account_age)
        political_word_ratio = political_word_total_word_ratio(text)
        tweets_per_day = TweetsPerDay(tweets_count, account_age)


        
        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['entities_in_text'].append(entities_in_text)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['is_reply'].append(is_reply)
        dfPolitical['source_counts'].append(source_counts)
        dfPolitical['account_activeness'].append(account_activeness)
        dfPolitical['account_age'].append(account_age)
        dfPolitical['political_word_ratio'].append(political_word_ratio)
        dfPolitical['digit_count'].append(digit_count)
        dfPolitical['tweet_length'].append(tweet_length)
        dfPolitical['avg_fav'].append(avg_fav)
        dfPolitical['tweets_per_day'].append(tweets_per_day)

In [ ]:
dfPolitical = pd.DataFrame(dfPolitical)   #['tweet_id', 'is_retweet', 'num_mentions',	'num_hashtags',	'num_retweets', 'num_favorites', 'user_id', 'entities_in_text', 'total_interactions', 'is_reply', 'account_activeness', 'political_word_ratio']
dfPolitical

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,entities_in_text,total_interactions,is_reply,source_counts,account_activeness,account_age,political_word_ratio,digit_count,tweet_length,avg_fav,tweets_per_day
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,0,Android,1.379707,2017-10-19 10:43:55+00:00,0.000000,0,140,18.657427,1.379707
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,0,0,1,Android,1.713586,2012-05-31 14:08:26+00:00,0.000000,0,97,0.154937,1.713586
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,iPhone,4.118310,2021-02-01 22:56:06+00:00,0.000000,0,140,1.732394,4.118310
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,1,Other,4.780161,2022-01-05 17:08:49+00:00,0.000000,0,140,1.630027,4.780161
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,0,Android,26.900356,2022-04-07 07:58:42+00:00,0.000000,8,140,8.637011,26.900356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,0,90,0,Other,10.600513,2012-05-10 11:47:23+00:00,0.000000,0,140,1.279487,10.600513
33529,1584027427696959488,0,None,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,2,9,1,iPhone,11.435438,2010-07-03 07:58:43+00:00,0.111111,2,140,42.197509,11.435438
33530,1585945783307730945,0,None,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,,2,1,1,Android,0.300687,2011-11-17 20:15:55+00:00,0.285714,0,76,0.348061,0.300687
33531,1569748909521801221,1,muazzezeralp,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,,2,6,0,Android,324.512658,2021-09-26 13:53:55+00:00,0.181818,0,140,92.805907,324.512658


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [ ]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']  
    user_is_verified = user_metadata_line['verified']
    user_has_default_photo = user_metadata_line['default_profile_image']
    user_statuses_count = user_metadata_line['statuses_count']

    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count, 
     'user_statuses_count': user_statuses_count, 'user_is_verified':user_is_verified, 'user_has_default_photo': user_has_default_photo}

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [ ]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [ ]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [ ]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'user_statuses_count':[],
         'description_len':[],
         'followers_to_all_ratio':[],
         'user_is_verified':[],
         'user_has_default_photo': []
         }

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [ ]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,2551,7,0.260000,False,False
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,42771,19,0.732260,False,False
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,14300,64,0.192308,False,False
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,21303,65,0.325203,False,False
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,1629,23,0.505051,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,2396,0,0.513453,False,False
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,75178,100,0.975088,False,False
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,6482,28,0.451362,False,False
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,121113,116,0.314431,False,False


### 1.2.2. Get Tweet Info of Users in user_tweeets.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [ ]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [ ]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [ ]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 1000 == 0:
            print(i)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [ ]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets
0,594642154,0.115000,2.0,200
1,525600289,0.005025,1.0,199
2,931895965501534209,0.900000,0.0,200
3,1591543462746329088,0.185000,0.0,200
4,734801354749796352,1.000000,0.0,200
...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200
28311,1475272459616235525,0.825000,0.0,200
28312,1096753792731750401,0.051020,1.0,196
28313,1269527617687953409,0.095000,2.0,200


### 1.2.3. Merge dfBot and dfBotTweets

In [ ]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo,retweet_total_ratio,num_median_favorites,num_of_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,2551,7,0.260000,False,False,0.395939,0.0,197.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,42771,19,0.732260,False,False,0.125000,0.0,200.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,14300,64,0.192308,False,False,0.910000,0.0,200.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,21303,65,0.325203,False,False,0.015306,1.0,196.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,1629,23,0.505051,False,False,0.659898,0.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,2396,0,0.513453,False,False,0.015000,1.0,200.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,75178,100,0.975088,False,False,0.291457,2.0,199.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,6482,28,0.451362,False,False,0.061538,0.0,195.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,121113,116,0.314431,False,False,0.995000,0.0,200.0


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [ ]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train.head()


,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,entities_in_text,total_interactions,is_reply,source_counts,account_activeness,account_age,political_word_ratio,digit_count,tweet_length,avg_fav,tweets_per_day,isPolitical,topics
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,0,Android,1.379707,2017-10-19 10:43:55+00:00,0.0,0,140,18.657427,1.379707,No,['Education']
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,0,0,1,Android,1.713586,2012-05-31 14:08:26+00:00,0.0,0,97,0.154937,1.713586,No,['Religion']
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,iPhone,4.118310,2021-02-01 22:56:06+00:00,0.0,0,140,1.732394,4.118310,No,['Nationalsecurity']
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,1,Other,4.780161,2022-01-05 17:08:49+00:00,0.0,0,140,1.630027,4.780161,Yes,['Foreignpolicy']
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,0,Android,26.900356,2022-04-07 07:58:42+00:00,0.0,8,140,8.637011,26.900356,No,['Unemployment']


### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [ ]:
#X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags']]
#y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

####New Baseline Model

In [ ]:
X = dfPoliticalAll_train[['tweet_id', 'is_retweet', 'num_mentions',	'num_hashtags',	'num_retweets', 'num_favorites', 'entities_in_text', 'total_interactions', 'is_reply', 'account_activeness', 'political_word_ratio', 'digit_count', 'tweet_length', 'tweets_per_day', 'avg_fav']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)


### 2.1.3. Train - validation split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# specify the parameter grid
param_grid = {'max_depth': [5, 10, 20, 30],
              'min_samples_leaf': [10, 50, 100, 110, 130, 150]}

# create an instance of the DecisionTreeClassifier
dtc_tweet = DecisionTreeClassifier()

# create an instance of the GridSearchCV
grid_search_tweet = GridSearchCV(dtc_tweet, param_grid, cv=5, scoring='neg_mean_squared_error')

# fit the grid search to the training data
grid_search_tweet.fit(X_train, y_train)

# print the best parameters
print("Best parameters: ", grid_search_tweet.best_params_)

# print the MSE for the best parameters
print("MSE for best parameters: ", -grid_search_tweet.best_score_)


Best parameters:  {'max_depth': 5, 'min_samples_leaf': 100}
MSE for best parameters:  0.1640625


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_political = DecisionTreeClassifier()

# fit your model
dtc_political.fit(X_train, y_train)

# make predictions
preds = dtc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.303125 
 Accuracy Score: 0.696875 
 Confusion Matrix: 
 [[128  88]
 [106 318]]


#### 2.1.4.1 Model Training (RFC)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error


# create an instance of the classifier
rfc_political = RandomForestClassifier()

# fit the model to the training data
rfc_political.fit(X_train, y_train)

# make predictions on the validation set
preds = rfc_political.predict(X_valid)

# evaluate the model on the validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)





MSE: 0.171875 
 Accuracy Score: 0.828125 
 Confusion Matrix: 
 [[139  77]
 [ 33 391]]


#### 2.1.4.2. Model Training (Linear Regression)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# calculate the MSE
mse = mean_squared_error(y_valid, preds)

# calculate the MAE
mae = mean_absolute_error(y_valid, preds)

# calculate the R2 score
r2 = r2_score(y_valid, preds)

print("MSE:", mse, "\n",
      "MAE:", mae, "\n",
      "R2 Score:", r2)


MSE: 0.171875 
 MAE: 0.171875 
 R2 Score: 0.23130677847658987


#### 2.1.4.3. Model Training (AdaBoost)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance of the classifier
adaboost_political = AdaBoostClassifier()

# fit the model to the training data
adaboost_political.fit(X_train, y_train)

adaboost_political.base_estimator_

# make predictions on the validation set
preds = adaboost_political.predict(X_valid)



# evaluate the model on the validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
    "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


MSE: 0.1640625 
 Accuracy Score: 0.8359375 
 Confusion Matrix: 
 [[140  76]
 [ 29 395]]


#### 2.1.4.4. Model Training (K-Nearest Neighbors)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance of the classifier
knn_political = KNeighborsClassifier()

# fit the model to the training data
knn_political.fit(X_train, y_train)

# make predictions on the validation set
preds = knn_political.predict(X_valid)

# evaluate the model on the validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
    "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


MSE: 0.1828125 
 Accuracy Score: 0.8171875 
 Confusion Matrix: 
 [[136  80]
 [ 37 387]]


#### ENSEMBLE METHOD

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

# Create an instance of the AdaBoost classifier
ada_classifier = AdaBoostClassifier()

# Create an instance of the Random Forest classifier
knn_classifier = RandomForestClassifier()

# Create an instance of the Voting Classifier
ensemble_classifier = VotingClassifier(estimators=[('ada', ada_classifier), ('knn', knn_classifier)], voting='hard')

# Fit the ensemble classifier to the training data
ensemble_classifier.fit(X_train, y_train)

# make predictions on the validation set
preds = ensemble_classifier.predict(X_valid)

# evaluate the model on the validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


MSE: 0.1828125 
 Accuracy Score: 0.828125 
 Confusion Matrix: 
 [[145  71]
 [ 39 385]]


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [ ]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower() #['user_followers_count', 'user_friends_count', 'user_statuses_count', 'description_len', 'followers_to_all_ratio', 'user_has_default_photo', 'retweet_total_ratio', 'num_median_favorites', 'num_of_tweets']

In [ ]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot,gender
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,2551,7,0.260000,False,False,0.395939,0.0,197.0,nasreenakhan006,No,Male
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,42771,19,0.732260,False,False,0.125000,0.0,200.0,scorpiehoez,Yes,Female
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,14300,64,0.192308,False,False,0.910000,0.0,200.0,yusufak63712920,Yes,Male
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,21303,65,0.325203,False,False,0.015306,1.0,196.0,nedenburdaysam,No,Female
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,1629,23,0.505051,False,False,0.659898,0.0,197.0,biologselim,No,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,5288,0,0.712264,False,False,0.000000,1.0,200.0,anka6054,No,NaN
3196,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,311,0,0.320370,False,False,0.580000,0.0,200.0,atamabekleyenzz,No,NaN
3197,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,1029,71,0.498215,False,False,0.040000,36.0,200.0,memrahinci,No,NaN
3198,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,18,26,0.090909,False,False,0.769231,0.0,13.0,muratkkk18,No,NaN


In [ ]:
trainingUserDf.isBot.value_counts()

No          2577
Yes          617
Not sure       6
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

NEW BASELINE MODEL

In [ ]:
X = dfBotAll_train[['user_followers_count', 'user_friends_count', 'user_statuses_count', 'description_len', 'followers_to_all_ratio', 'user_has_default_photo', 'retweet_total_ratio', 'num_median_favorites']]
y = dfBotAll_train['isBot'].apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_bot = DecisionTreeClassifier()

# fit your model
dtc_bot.fit(X_train, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.1828125 
 Accuracy Score: 0.6921875 
 Confusion Matrix: 
 [[405  95]
 [102  38]]


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# specify the parameter grid
param_grid = {'max_depth': [5, 10, 20],
              'min_samples_leaf': [10, 50, 100]}

# create an instance of the DecisionTreeClassifier
dtc_bot = DecisionTreeClassifier()

# create an instance of the GridSearchCV
grid_search = GridSearchCV(dtc_bot, param_grid, cv=5, scoring='neg_mean_squared_error')

# fit the grid search to the training data
grid_search.fit(X_train, y_train)

# print the best parameters
print("Best parameters: ", grid_search.best_params_)

# print the MSE for the best parameters
print("MSE for best parameters: ", -grid_search.best_score_)


Best parameters:  {'max_depth': 5, 'min_samples_leaf': 10}
MSE for best parameters:  0.188671875


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# specify the parameter grid
param_grid = {'max_depth': [1, 5, 10, 20, 30],
              'min_samples_leaf': [1, 10, 50, 100, 200],
              'min_samples_split':[2,5,10],
              'max_leaf_nodes':[2,5,10,20],
              'criterion':['entropy', 'gini']}

# create an instance of the DecisionTreeClassifier
dtc_bot2 = DecisionTreeClassifier()

# fit the DecisionTreeClassifier to the training data
dtc_bot2.fit(X_train, y_train)

# create an instance of the GridSearchCV
bot_grid_search = GridSearchCV(dtc_bot2, param_grid, cv=5, scoring='neg_mean_squared_error')

# fit the grid search to the training data
bot_grid_search.fit(X_train, y_train)

# print the best parameters
print("Best parameters: ", bot_grid_search.best_params_)

# print the MSE for the best parameters
print("MSE for best parameters: ", -bot_grid_search.best_score_)


Best parameters:  {'criterion': 'entropy', 'max_depth': 5, 'max_leaf_nodes': 20, 'min_samples_leaf': 50, 'min_samples_split': 2}
MSE for best parameters:  0.182421875


#### (NEW) 2.2.4.1. Model Training (AdaBoost) 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance of the classifier
adaBoost_bot = AdaBoostClassifier()

# fit the model to the training data
adaBoost_bot.fit(X_train, y_train)

# make predictions on the validation set
preds = adaBoost_bot.predict(X_valid)

# evaluate the model on the validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
    "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


MSE: 0.1828125 
 Accuracy Score: 0.7921875 
 Confusion Matrix: 
 [[479  21]
 [112  28]]


#### (NEW3) 2.2.4.3. Model Training (RFC)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance
rfc_bot = RandomForestClassifier(random_state = 42)

# Fit the model
rfc_bot.fit(X_train, y_train)

# Make predictions
preds = rfc_bot.predict(X_valid)

# Evaluate the model on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


Accuracy Score: 0.7828125 
 Confusion Matrix: 
 [[479  21]
 [118  22]]


In [ ]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np

# specify the parameter grid
param_grid = {'max_depth': range(3, 8, 2),
              'min_child_weight':range(1,4,2),
              'subsample':[i/10.0 for i in range(6,10)],
              'colsample_bytree':[i/10.0 for i in range(6,10)],
              'gamma':[i/10.0 for i in range(0,3)],
              'n_estimators': [50, 100, 200]
              }

# create an instance of the XGBoost regressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=4, early_stopping_rounds=5)
xgb_reg.fit(X_train, y_train)  # fit the model to the training data

random_search_bot = RandomizedSearchCV(xgb_reg, param_grid, cv=5, n_iter=20, scoring='neg_mean_squared_error', verbose=True)
random_search_bot.fit(X_train, y_train)  # fit the random search to the training data

print("Best parameters: ", random_search_bot.best_params_)

print("MSE for best: ", -random_search_bot.best_score_)

random_bot = random_search_bot.predict(X_valid)

random_bot = np.round(random_bot)

# evaluate on validation set
mse = mean_squared_error(y_valid, random_bot)

print("MSE:", mse)


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [ ]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round2-tweet.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()


# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPolitical_test[['tweet_id', 'is_retweet', 'num_mentions',	'num_hashtags',	'num_retweets', 'num_favorites', 'entities_in_text', 'total_interactions', 'is_reply', 'account_activeness', 'political_word_ratio', 'digit_count', 'tweet_length', 'tweets_per_day', 'avg_fav']]

# make predictions based on these variables
predictions_political = grid_search_tweet.predict(X)

### This part is important! We expect you to return your predictions in the following format:

In [ ]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1593649159009099777': 1.0,
 '1434787703783051264': 0.0,
 '1427339600083791885': 0.0,
 '1388235183653011462': 0.0,
 '1579558096833511424': 1.0,
 '1592120408073203712': 1.0,
 '1439547067337256967': 0.0,
 '1577024342268837888': 1.0,
 '1365726640416976906': 0.0,
 '1597274845381029888': 1.0,
 '1597512443328167936': 1.0,
 '1415032260571680768': 0.0,
 '1583477966373543936': 1.0,
 '1564926450096013313': 1.0,
 '1597138789108895744': 1.0,
 '1384499047390658560': 0.0,
 '1596583748669419521': 1.0,
 '1391681495622995971': 0.0,
 '1365710259549966339': 0.0,
 '1590673118397624323': 1.0,
 '1597256187325878273': 1.0,
 '1588464678443024385': 1.0,
 '1413108476348354562': 0.0,
 '1579408398894137344': 1.0,
 '1570758749606019073': 1.0,
 '1588936302086737925': 1.0,
 '1596233602886701057': 1.0,
 '1584922292127256577': 1.0,
 '1586279180983042050': 1.0,
 '1434963357023473671': 0.0,
 '1588911636126920704': 1.0,
 '1399687111234756612': 0.0,
 '1363226564683522051': 0.0,
 '1596978345560150016': 1.0,
 '159368650662

## 3.2. Predictions for Users (Bot or Not)

In [ ]:
evaluationUserDf = pd.read_csv('{}evaluation-round2-user.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBotAll_train[['user_followers_count', 'user_friends_count', 'user_statuses_count', 'description_len', 'followers_to_all_ratio', 'user_has_default_photo', 'retweet_total_ratio', 'num_median_favorites']]

# make predictions based on these variables
predictions_bot = bot_grid_search.predict(X)

In [ ]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'nedenburdaysam': 0.0,
 'biologselim': 0.0,
 'bilgin21604923': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mustafaarst': 0.0,
 'mvnez': 0.0,
 'farukhalit2': 0.0,
 'harlunoshi': 1.0,
 'tamerduran_1': 0.0,
 'donkisotumsu': 0.0,
 'enveraysevera': 1.0,
 'gendenmukatol': 0.0,
 '1905anason': 1.0,
 'dasiskein': 0.0,
 'ercan_bas29': 0.0,
 'mett_1907': 0.0,
 'haberinyokcokk': 0.0,
 'han34nesli': 0.0,
 'mehmetaltay64': 0.0,
 'nurtencam2': 0.0,
 'berkeduranovic': 1.0,
 'cagdasadim': 1.0,
 'yorumsuzadam87': 0.0,
 'twitsildiren': 1.0,
 'nurdiyoloji': 0.0,
 'merabalare': 0.0,
 'sevdaac72373936': 0.0,
 'cakan0_': 0.0,
 'ozlemesor': 0.0,
 'muratyazgan51': 0.0,
 'emre_caliskann': 0.0,
 'oguzhanonel': 0.0,
 'mintchocolatemx': 0.0,
 'mehmet07454846': 0.0,
 'lifegs': 0.0,
 'begumkarabeyx': 0.0,
 'mhakankocak': 1.0,
 'avutulan': 0.0,
 'imamgibiimam': 0.0,
 'durdane52': 0.0,
 'emsettinerdoan7': 1.0,
 'suaterce': 0.0,
 'ibrahimcalsskan': 0.0,
 'uykusuz75': 0.0,
 'goksubayy': 0.0,
 'samobisus': 0.

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [ ]:
# Explain your approach

data_explanations = '''
At the previous step i have tried to implement SMOTE, in this specific implementation the sampling_strategy parameter for both SMOTE and RandomUnderSampler is set to 0.6. 
This means that the resulting dataset will have a balance of 40% minority class samples and 60% majority class samples. After trying it on different model there were very small (0.01) positive impact on the result.
After trying new models for my data and adding new features i have seen that SMOTE did not show any improvement so i have decided not to use it and see a small boost (0.05).
'''

feature_explanations = '''
For feature extraction process, i have added more useful features which were not included in the base template model;
digit_count: Amount of digits used in a username
tweet_length: The length of a tweet
avg_fav: Average amount of tweets favourited by the user since account created.
tweets_per_day: Average amount of tweets tweeted by the user per day.
'''

model_explanations = '''
At the second round, I chose to use XGBoost. It has an ability to perform well with a large number of parameters. 
To optimize the performance of the model, I used a GridSearchCV or RandomizedSearchCV method to tune the hyperparameters. 
This method allows us to find the best set of parameters for the model, which can significantly improve its performance. 
It helped me get almost 0.7 improvement from the previous isBot result. This is the result of the code:
Best parameters:  {'subsample': 0.6, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 3, 'gamma': 0.1, 'colsample_bytree': 0.9}
MSE for best:  0.1327547545106528
MSE: 0.1471139803443762
However, results became float like this, "'malsfbittik': 0.4501497149467468". When i round the numbers MSE decreases to 0.20.
Thats why i used GridSearchCV and got almost 0.3 decrease in MSE which is: 0.182421875

Additionally for the isPolitical part, i have used the new features to train however i got 0.05 reduction in MSE. 
Adaboost and GridSearchCV gave very close results and it is around 0.1604 while it was 0.158 before. 
'''

additional_explanations = '''
I chose to use GridSearchCV method to tune the hyperparameters of the XGBoost model in order to optimize its performance. 
However, I realized that the GridSearchCV method took a significant amount of time to give an output due to the large number of combinations of parameters that it had to try.
In order to overcome this limitation, I decided to switch to the RandomizedSearchCV method instead of GridSearchCV. 
The RandomizedSearchCV method randomly samples a set of parameters from the specified parameter grid, which reduces the number of iterations and the overall computation time.
'''


In [ ]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [ ]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'round': 2,
 'student_id': '26555',
 'user_predictions': {'nedenburdaysam': 0.0,
  'biologselim': 0.0,
  'bilgin21604923': 0.0,
  'denizlihabercom': 0.0,
  'burakerbaychp': 0.0,
  'mustafaarst': 0.0,
  'mvnez': 0.0,
  'farukhalit2': 0.0,
  'harlunoshi': 1.0,
  'tamerduran_1': 0.0,
  'donkisotumsu': 0.0,
  'enveraysevera': 1.0,
  'gendenmukatol': 0.0,
  '1905anason': 1.0,
  'dasiskein': 0.0,
  'ercan_bas29': 0.0,
  'mett_1907': 0.0,
  'haberinyokcokk': 0.0,
  'han34nesli': 0.0,
  'mehmetaltay64': 0.0,
  'nurtencam2': 0.0,
  'berkeduranovic': 1.0,
  'cagdasadim': 1.0,
  'yorumsuzadam87': 0.0,
  'twitsildiren': 1.0,
  'nurdiyoloji': 0.0,
  'merabalare': 0.0,
  'sevdaac72373936': 0.0,
  'cakan0_': 0.0,
  'ozlemesor': 0.0,
  'muratyazgan51': 0.0,
  'emre_caliskann': 0.0,
  'oguzhanonel': 0.0,
  'mintchocolatemx': 0.0,
  'mehmet07454846': 0.0,
  'lifegs': 0.0,
  'begumkarabeyx': 0.0,
  'mhakankocak': 1.0,
  'avutulan': 0.0,
  'imamgibiimam': 0.0,
  'durdane52': 0.0,
  'emsettinerdoan7': 1.0